# (Re-)run Hugarian algorithm

> "Because it stops after a few hundred"

- branch: master
- hidden: true
- categories: [hsi, munkres]

In [225]:
import json

with open("/tmp/procced2.1.json") as inf:
    a = json.load(inf)

In [226]:
filtered1 = [x for x in a if not "discarded" in x]

In [227]:
already_done = []
with open("/tmp/assignment_short.csv") as tsvf:
    for line in tsvf:
        line = line.strip()
        if line.startswith("syntitem"):
            continue
        if not line:
            continue
        already_done.append(line.split(",")[0])

In [228]:
filtered = [x for x in filtered1 if not x["fileid"] in already_done]

In [229]:
len(filtered), len(filtered1), len(already_done)

(423, 1412, 989)

In [230]:
import numpy as np

L = np.load('bvh_pt_lengths.npy',allow_pickle=True)
framerate = 120
point_length = {}
for thing in L:
    item = list(thing.keys())[0].split('/')[-1]
    item = item.replace('.bvh','')
    point_length[item] = list(thing.values())[0]/framerate

In [231]:
synth_length = {}
for item in filtered:
    synth_length[item["fileid"]] = item["duration"]

In [232]:
# get Anna's names from my list
synth_pre = {}
synth_post = {}
synth_data = {}
synth_times = []

for item in filtered:
    fileid = item["fileid"]
    dem_start = float(item["determiner_start"])
    dem_end = float(item["determiner_end"])
    duration = float(item["duration"])

    synth_pre[fileid] = dem_start
    synth_post[fileid] = dem_end - dem_start
    synth_data[fileid] = (dem_start, duration)
    synth_times.append((duration, dem_start))

In [233]:
import pandas as pd
from glob import glob

def find_demonstrative_index(expression, demonstratives):
    words = expression.split(' ')
    # Iterate over each word to find if it matches any of the demonstratives
    for i, word in enumerate(words):
        if word in demonstratives:
            return i  # Return the index of the first matching demonstrative
    return -1  # Return -1 if no demonstratives are found
demonstratives = ['this','that','one','those','these','there','here']

files = glob('/tmp/tsv_pt_segments/*.tsv')
print(len(files))
words_to_exclude = ['left', 'right', 'middle', 'back']
files = [f for f in files if not any(word in f for word in words_to_exclude)]
pt_times = []
pt_names = []
pt_pre = {}
pt_post = {}
pt_data = {}
for fn in files:
    temp_list = []
    item = fn.split('/')[-1].split('.tsv')[0]
    with open(fn) as f:
        with open(fn) as f:
            for line in f:
                t0, t1, wrd = line.strip().split('\t')
                t0, t1 = float(t0), float(t1)
                temp_list.append([t0, t1, wrd])
        df = pd.DataFrame(temp_list, columns=['t0','t1','wrd'])
        expr = ' '.join(df['wrd'])
        index  = find_demonstrative_index(expr, demonstratives)
        dem_time = df['t0'].iloc[index]
        total_time = point_length[item]
        pt_times.append((total_time, dem_time))
        pt_names.append(item)
        pt_pre[item] = dem_time
        pt_post[item] = point_length[item] - pt_pre[item]
        pt_data[item] = (dem_time,total_time)


1147


In [235]:
# Convert the list to a numpy array for statistical calculations
synth_lengths_array = np.array([x[0] for x in synth_times])
print(synth_lengths_array)
lengths_array = np.array(synth_lengths_array)

lengths_array_pt = np.array([x[0] for x in pt_times])
# Calculate the mean and standard deviation
mean_length = np.mean(lengths_array)
std_dev_length = np.std(lengths_array)

# Calculate the 50th (median) and 75th percentiles
median_length_syn = np.percentile(lengths_array, 50)
percentile_75_length = np.percentile(lengths_array, 75)
percentile_75_length

cutoff_time_syn = median_length_syn

median_length_pt = np.percentile(lengths_array_pt, 50)
percentile_75_length = np.percentile(lengths_array, 75)
percentile_75_length

cutoff_time_pt = median_length_pt

print(len(pt_data))
print(len(synth_data))

short_pt = {k:v for k,v in pt_data.items() if v[1]<cutoff_time_pt}
short_synth = {k:v for k,v in synth_data.items() if v[1]<cutoff_time_syn}
long_pt = {k:v for k,v in pt_data.items() if v[1]>=cutoff_time_pt}
long_synth = {k:v for k,v in synth_data.items() if v[1]>=cutoff_time_syn}

synth_pre_short = {k:v for k,v in synth_pre.items() if k in short_synth}
synth_post_short = {k:v for k,v in synth_post.items() if k in short_synth}
pt_pre_short = {k:v for k,v in pt_pre.items() if k in short_pt}
pt_post_short = {k:v for k,v in pt_post.items() if k in short_pt}



[ 4.576       5.06666667  4.59733333  5.61066667  5.36533333  6.73066667
  5.024       4.96        7.37066667  6.92266667  7.21066667  4.59733333
  5.28        4.59733333  5.61066667  6.51733333  5.55733333  5.01333333
  5.86666667  5.26933333  6.34666667  6.15466667  6.61333333  6.208
  6.12266667  5.71733333  4.992       4.512       4.91733333  5.81333333
  5.29066667  6.79466667  5.42933333  5.376       5.16266667  5.48266667
  4.66133333  5.984       5.48266667  5.86666667  6.02666667  5.38666667
  6.73066667  7.34933333  8.288       5.29066667  4.61866667  5.312
  5.29066667  5.12        4.65066667  7.18933333  4.98133333  4.736
  4.736       9.32266667  4.704       4.768       4.78933333  4.59733333
  5.14133333  5.248       5.92        4.74666667  5.056       5.51466667
  4.87466667  5.70666667  5.06666667  4.75733333  4.832       4.736
  5.86666667  5.30133333  5.84533333  5.22666667  4.8        10.12266667
  5.01333333  5.472       5.024       4.98133333  5.32266667  4.6826666

In [236]:
# make distance matrix
D = []
synth_pre = synth_pre_short
synth_post = synth_post_short
pt_pre = pt_pre_short
pt_post = pt_post_short

for syntitem in list(synth_pre.keys())[:]:
    row = []
    for pointitem in list(pt_pre.keys())[:]:
        syntpre = synth_pre[syntitem]
        syntpost = synth_post[syntitem]
        pointpre = pt_pre[pointitem]
        pointpost = pt_post[pointitem]

        cost = abs(syntpre-pointpre) + abs(syntpost-pointpost)
        if synth_length[syntitem] > point_length[pointitem]:
            cost *= 10
        # penalize if synt starts before or ends after point
        if syntpre > pointpre:
            cost *= 2
        if syntpost > pointpost:
            cost *= 2
        row.append(cost)
    
    D.append(row)
dd = np.array(D)
dd.shape

(210, 573)

In [187]:
!pip install munkres

  Using cached munkres-1.1.4-py2.py3-none-any.whl (7.0 kB)


In [224]:
from munkres import Munkres
m = Munkres()
assignment_re = m.compute(D)
f = open('assignment_short.csv','w')
f.write('syntitem,pointitem,offset\n')
for pair in assignment_re:
    syntidx,pointidx = pair
    syntitem = list(synth_pre.keys())[syntidx]
    pointitem = list(pt_pre.keys())[pointidx]
    f.write('{},{},{}\n'.format(syntitem, pointitem, pt_pre[pointitem]-synth_pre[syntitem]))
f.close()

In [193]:
selected = []
with open("/tmp/assignment_short.csv") as f:
    for line in f.readlines():
        line = line.strip()
        if line.startswith("syntitem"):
            continue
        parts = line.split(",")
        selected.append(parts[0])

In [195]:
filtered = []
with open("/tmp/procced2.1.json") as inf:
    newdata = json.load(inf)
for item in newdata:
    if item["fileid"] in selected:
        filtered.append(item)

In [198]:
# ffmpeg -i /tmp/output/st2_mirrored.10512_10854_1.23stretch.mp4 -i groundinggpt-generated-speech/hsi_4_0717_211_001_main__ggpt__682.wav -filter_complex "[1:a]atrim=start=0.8624999999999999[aud]" -map 0:v -map "[aud]" -c:v copy -c:a aac output_minus.mp4

with open("/tmp/assignment_short.csv") as f, open("/tmp/ffmpeg-runner.sh", "w") as outf:
    for line in f.readlines():
        line = line.strip()
        if line.startswith("syntitem"):
            continue
        parts = line.split(",")
        
        outf.write(f"ffmpeg -i /tmp/output/{parts[1]}.mp4 -i /tmp/groundinggpt-generated-speech/{parts[0]}.wav")
        time = float(parts[2])
        if time < 0.0:
            outf.write(f" -filter_complex \"[1:a]atrim=start={time}[aud]\" -map 0:v -map \"[aud]\"")
        else:
            itime = int(time * 1000.0)
            outf.write(f" -filter_complex \"[1:a]adelay={itime}|{itime}[aud]\" -map 0:v -map \"[aud]\"")
        outf.write(f" -c:v copy -c:a aac /tmp/output_minus/{parts[0]}.mp4\n")


In [201]:
filtered_dict = {x["fileid"]: x for x in filtered}

In [203]:
with open("/tmp/assignment_short.csv") as f, open("/tmp/assigned_extended.csv", "w") as outf:
    for line in f.readlines():
        line = line.strip()
        if line.startswith("syntitem"):
            outf.write(line + ",room,topic\n")
            continue
        parts = line.split(",")
        cur = filtered_dict[parts[0]]
        outf.write(f"{line},{cur['room']},{cur['topic']}\n")
